In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [4]:
path_to_file = '..\Lesson9\evgenyi_onegin (2).txt'

In [11]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [7]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [12]:
# text = text + text
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [13]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [21]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [27]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(3):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '


In [28]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [36]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [37]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [38]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [39]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 128)           16768     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          4722688   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 131)           134275    
Total params: 30,051,843
Trainable params: 30,051,843
Non-trainable params: 0
____________________________________________

In [45]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [83]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=50,
    save_weights_only=True)

In [84]:
checkpoint_prefix

'./training_checkpoints\\ckpt_{epoch}'

In [85]:
model.compile(optimizer='adam', loss=loss)

In [86]:
EPOCHS = 300

In [87]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/300
44/44 [==============================] - 10s 148ms/step - loss: 0.2263
Epoch 2/300
44/44 [==============================] - 7s 152ms/step - loss: 0.1986
Epoch 3/300
44/44 [==============================] - 7s 154ms/step - loss: 0.1968
Epoch 4/300
44/44 [==============================] - 7s 158ms/step - loss: 0.1997
Epoch 5/300
44/44 [==============================] - 7s 159ms/step - loss: 0.1912
Epoch 6/300
44/44 [==============================] - 7s 159ms/step - loss: 0.1869
Epoch 7/300
44/44 [==============================] - 7s 158ms/step - loss: 0.1879
Epoch 8/300
44/44 [==============================] - 7s 157ms/step - loss: 0.2129
Epoch 9/300
44/44 [==============================] - 7s 158ms/step - loss: 0.1891
Epoch 10/300
44/44 [==============================] - 7s 161ms/step - loss: 0.1844
Epoch 11/300
44/44 [==============================] - 7s 161ms/step - loss: 0.1787
Epoch 12/300
44/44 [==============================] - 7s 164ms/step - loss: 0.1757
Epoch 13/300

44/44 [==============================] - 7s 158ms/step - loss: 0.1301
Epoch 100/300
44/44 [==============================] - 7s 160ms/step - loss: 0.1420
Epoch 101/300
44/44 [==============================] - 7s 157ms/step - loss: 0.1367
Epoch 102/300
44/44 [==============================] - 7s 158ms/step - loss: 0.1261
Epoch 103/300
44/44 [==============================] - 7s 165ms/step - loss: 0.1240
Epoch 104/300
44/44 [==============================] - 7s 159ms/step - loss: 0.1224
Epoch 105/300
44/44 [==============================] - 7s 160ms/step - loss: 0.1226
Epoch 106/300
44/44 [==============================] - 7s 154ms/step - loss: 0.1305
Epoch 107/300
44/44 [==============================] - 7s 149ms/step - loss: 0.1407
Epoch 108/300
44/44 [==============================] - 7s 149ms/step - loss: 0.1250
Epoch 109/300
44/44 [==============================] - 7s 149ms/step - loss: 0.1233
Epoch 110/300
44/44 [==============================] - 7s 158ms/step - loss: 0.1983
Epoch 

44/44 [==============================] - 7s 154ms/step - loss: 0.1075
Epoch 197/300
44/44 [==============================] - 7s 154ms/step - loss: 0.1071
Epoch 198/300
44/44 [==============================] - 7s 154ms/step - loss: 0.1076
Epoch 199/300
44/44 [==============================] - 7s 154ms/step - loss: 0.1069
Epoch 200/300
44/44 [==============================] - 7s 154ms/step - loss: 0.1088
Epoch 201/300
44/44 [==============================] - 7s 155ms/step - loss: 0.1084
Epoch 202/300
44/44 [==============================] - 7s 154ms/step - loss: 0.1108
Epoch 203/300
44/44 [==============================] - 7s 154ms/step - loss: 0.1111
Epoch 204/300
44/44 [==============================] - 7s 155ms/step - loss: 0.1099
Epoch 205/300
44/44 [==============================] - 7s 155ms/step - loss: 0.1066
Epoch 206/300
44/44 [==============================] - 7s 155ms/step - loss: 0.1056
Epoch 207/300
44/44 [==============================] - 7s 155ms/step - loss: 0.1077
Epoch 

44/44 [==============================] - 7s 151ms/step - loss: 0.0975
Epoch 294/300
44/44 [==============================] - 7s 148ms/step - loss: 0.0991
Epoch 295/300
44/44 [==============================] - 7s 148ms/step - loss: 0.1009
Epoch 296/300
44/44 [==============================] - 7s 149ms/step - loss: 0.1013
Epoch 297/300
44/44 [==============================] - 7s 149ms/step - loss: 0.1031
Epoch 298/300
44/44 [==============================] - 7s 160ms/step - loss: 0.1042
Epoch 299/300
44/44 [==============================] - 7s 156ms/step - loss: 0.1041
Epoch 300/300
44/44 [==============================] - 7s 156ms/step - loss: 0.1068


In [88]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_300'

In [89]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [102]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.9

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [103]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже темно,
                        Стоит подобить вечера тогда
                        К плечу головушкой склонилась,
                                      Уловку выдумав и грудь
                        Отворотяв от педант и сладкий плод.
                        Но в возраст поздний и бесплодный,
                                                          XXXVII

                        Вот, окружен своей дубравой.
                        "Но, что с вами игракать
                        Дрожащей ручко
